In [1]:
from json import loads
import pandas as pd
import numpy as np

In [2]:
song_scraped= loads(open("artists_scraped.json").read())
BB = pd.read_csv("data/Hot Stuff.csv")

In [3]:
print("No. Total Artists:    ", len(set(BB["Performer"])))
print("No. Artists Scraped:  ", len(song_scraped))
print("No. Missing Artists:  ", len(set(BB["Performer"]) - set(song_scraped.keys())))

No. Total Artists:     9447
No. Artists Scraped:   4523
No. Missing Artists:   4924


### Examine Commonality Between The Missing Artists

In [4]:
from collections import Counter

In [5]:
missing_artists = list(set(BB["Performer"]) - set(song_scraped.keys()))
def get_count(ls):
    bag_of_artists_name = list()
    for i in ls:
        bag_of_artists_name += i.split(" ")
    name_counts = Counter(bag_of_artists_name)
    print(name_counts.most_common(20))
get_count(missing_artists)

[('Featuring', 1930), ('&', 1198), ('The', 952), ('And', 250), ('Lil', 181), ('With', 167), ('Wayne', 108), ('Brown', 90), ('Chris', 87), ('and', 84), ('Band', 84), ('Drake', 80), ('Nicki', 74), ('His', 71), ('Young', 70), ('Minaj', 69), ('Orchestra', 69), ('Big', 68), ('Kanye', 64), ('West', 61)]


In [6]:
missing_artists_reformat = list(missing_artists)
for i in range(len(missing_artists)):
    if "Featuring" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("Featuring")[0].strip()
    if "Feat." in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("Feat.")[0].strip()
    if "featuring" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("featuring")[0].strip()
    if "&" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("&")[0].strip()
    if "and" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("and")[0].strip()
    if "And" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("And")[0].strip()
    if "With" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("With")[0].strip()
    if "with" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("with")[0].strip()
    if "(" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("(")[0].strip()
get_count(missing_artists_reformat)

[('The', 569), ('B', 71), ('Lil', 66), ('Brown', 41), ('David', 40), ('John', 39), ('DJ', 37), ('Of', 36), ('Chris', 36), ('Drake', 35), ('Joe', 35), ('Wayne', 35), ('Big', 35), ('Paul', 34), ('Bobby', 33), ('Young', 29), ('Johnny', 28), ('Kanye', 28), ('Sean', 27), ('West', 26)]


In [7]:
import requests
import re
from random import randrange
from bs4 import BeautifulSoup
from collections import defaultdict

In [56]:
import multiprocessing


comm = r'<table style="padding:2px; border-spacing:0px; border:0px; width:100%">\n<tr>\n<td style="width:50%; vertical-align:top;"><b class="text-yellow">Song<\/b><\/td>\n<td style="width:50%; vertical-align:top;"><b class="text-yellow">in Album<\/b><\/td><td>&nbsp;<\/td>\n<td>&nbsp;<\/td>\n<\/tr>\n([\S\s]*)\n<\/table>'
def scrape_data(i):
    ex = missing_artists_reformat[i]
    songs_array = []; album_array = []; url_array = []
    url = "https://lyrics.az/{}/allsongs.html"
    resp = requests.get(url.format("-".join(ex.lower().split())))
    m = re.search(comm, resp.text)
    if m:
        rows = m.group(1)
        soup = BeautifulSoup(rows, 'html.parser')
        for r in soup.find_all('tr'):
            find_link_album_song = r'<tr>\n<td style=\"vertical-align:top;\"><a href=\"(.*\.html)\"><div class=\"schemadiv\" itemscope=\"\" itemtype=\"http:\/\/schema\.org\/MusicRecording\">\n<span itemprop=\"name\">(.*)<\/span><\/div><\/a><\/td>\n<td style=\"vertical-align:top;\">\[(<a href=\".*\" style=\"color: #ffffff;\">(.*)<\/a>)?]<\/td>\n<td>\s<\/td>\n<td>\s<\/td>\n<\/tr>'
            #print("'{}'".format(str(r)))
            m = re.search(find_link_album_song, str(r))
            #print(m.group(1),m.group(2),m.group(4))
            songs_array.append(m.group(2))
            album_array.append(m.group(4))
            url_array.append("https://lyrics.az{}".format(m.group(1)))
        res = (missing_artists[i],{"Song": songs_array, "Album": album_array, "Url": url_array})
        return (True,res)
    else:
        return (False,missing_artists[i])


In [57]:
# should be running much faster 8 threads for each core, but if process gets stopped rip trying to restart

art_song_links = dict()
missing_artists_2 = set()
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*8) as pool:
    try:
        for suc,res in pool.imap_unordered(scrape_data, range(len(missing_artists))):
            if suc:
                art_song_links[res[0]] = res[1]
            else:
                missing_artists_2.add(res)
    except:
        print('Stopped search at index {}'.format(i))

In [58]:
len(art_song_links)

2551

In [59]:
len(missing_artists_2)+len(art_song_links) - len(missing_artists)

0

In [60]:
from json import dumps

In [61]:
jdata = dumps(art_song_links)
output_file = open("stripped_art_data.json","w+")
output_file.write(jdata)
output_file.close()

In [62]:
art_song = defaultdict(set)
for i,row in BB.iterrows():
    art_song[row['Performer']].add(row['Song'])
len(art_song)

9447

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15


In [63]:
art_song_links.keys()

dict_keys(['Florida Georgia Line Featuring Nelly', 'Cheat Codes Featuring Demi Lovato', 'Sylvia And Ralfi Pagan', 'Barry & The Tamerlanes', 'Patti Austin With James Ingram', "David Banner Featuring Lil' Flip", 'UGK Featuring OutKast', 'Ike & Tina Turner & The Ikettes', 'T-Pain Featuring Joey Galaxy', 'Spiral Starecase Featuring: Pat Upton', 'Maceo And The Macks', 'Public Announcement Featuring LeLe', 'Lil Yachty Featuring Ugly God', 'The Isley Brothers Featuring Angela Winbush', 'Eurythmics & Aretha Franklin', 'Usher Featuring Plies', 'Diana Ross & Michael Jackson', 'Dion & The Belmonts', 'Cashmere Cat Featuring Ariana Grande', 'Britney Spears Featuring Tinashe', 'Rae Sremmurd Featuring Nicki Minaj & Young Thug', 'Ace Hood Featuring Chris Brown', 'Roy Orbison And The Candy Men', 'Martina McBride With Jim Brickman', 'Enrique Iglesias Featuring Juan Luis Guerra', 'KeKe Wyatt Featuring Avant', 'Blake Shelton Featuring Gwen Sebastian', 'Kermit (Jim Henson)', 'Freddy Cannon & The Belmonts',

In [12]:
missing_artists

['Usher Featuring Plies',
 'Nicki Minaj, Drake & Lil Wayne',
 'Dion & The Belmonts',
 'Migos Featuring Travis Scott',
 "David Banner Featuring Lil' Flip",
 "Plain White T's",
 'Public Announcement Featuring LeLe',
 'American Flyer',
 'P$C Featuring T.I. & Lil Scrappy',
 'Rae Sremmurd Featuring Nicki Minaj & Young Thug',
 'Fabu',
 'Silkk The Shocker Featuring Mystikal',
 'Korona',
 'Nas Featuring will.i.am',
 'Dick Roman',
 'The Isley Brothers Featuring Angela Winbush',
 'Martina McBride With Jim Brickman',
 'Jadakiss Featuring Anthony Hamilton',
 'Rosco Martinez',
 'KeKe Wyatt Featuring Avant',
 'Britney Spears Featuring Tinashe',
 'Kendrick Lamar & SZA',
 'Aventura Featuring Don Omar',
 'Diana Ross & Michael Jackson',
 'Aaron Lewis Of Staind With Fred Durst',
 'Ace Hood Featuring Chris Brown',
 'Blake Shelton Featuring Gwen Sebastian',
 "The Edwin Hawkins' Singers Featuring Dorothy Combs Morrison",
 'Spiral Starecase Featuring: Pat Upton',
 'Shirley & Squirrely',
 'The Goodtimers',
 '

In [13]:
missing_artists_reformat

['Usher',
 'Nicki Minaj, Drake',
 'Dion',
 'Migos',
 'David Banner',
 "Plain White T's",
 'Public Announcement',
 'American Flyer',
 'P$C',
 'Rae Sremmurd',
 'Fabu',
 'Silkk The Shocker',
 'Korona',
 'Nas',
 'Dick Roman',
 'The Isley Brothers',
 'Martina McBride',
 'Jadakiss',
 'Rosco Martinez',
 'KeKe Wyatt',
 'Britney Spears',
 'Kendrick Lamar',
 'Aventura',
 'Diana Ross',
 'Aaron Lewis Of Staind',
 'Ace Hood',
 'Blake Shelton',
 "The Edwin Hawkins' Singers",
 'Spiral Starecase',
 'Shirley',
 'The Goodtimers',
 'Maceo',
 'Cashmere Cat',
 'Fire Inc.',
 'T-Pain',
 'Florida Georgia Line',
 'Kermit',
 "Michel'le",
 'Garl',
 '3rd Party',
 'Cheat Codes',
 '5 Satins',
 'Jon Thomas',
 'Cooper Brothers B',
 'Jose Jimenez',
 'Rihanna',
 'SoulDecision',
 'Pete Drake',
 'O.T. Genasis',
 'Nappy Roots',
 'Larry Groce',
 'Darrow Fletcher',
 'The Game',
 'Enrique Iglesias',
 'Sean Paul',
 'Leroy Pullins',
 'Patti Austin',
 'Joe Damiano',
 'Dean Parrish',
 'Lupe Fiasco',
 'The Hollyridge Strings',
 '